In [151]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64

In [174]:
df = pd.read_csv("data/healthcare-dataset-stroke-data.csv")
df.pop("stroke")

columns = df.columns.values
features = df.values[0]

inputs = {key: value for key, value in zip(columns, features)}
inputs

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...
5105,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [170]:
def string_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(
            value=[bytes(value, "utf-8")]
        ),
    )
    
def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(
            value=[value]
        ),
    )
    
def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(
            value=[value]
        ),
    )

In [171]:
def prepare_json(inputs: dict):
    feature_spec = dict()
    
    for keys, values in inputs.items():
        if isinstance(values, float):
            print(values)
            feature_spec[keys] = float_feature(values)
        elif isinstance(values, int):
            print(values)
            feature_spec[keys] = int_feature(values)
        elif isinstance(values, str):
            print(values)
            feature_spec[keys] = string_feature(values)
            
    print(feature_spec)
            
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
        
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    print(result)
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

In [172]:
def make_predictions(inputs):
    json_data = prepare_json(inputs)
    
    endpoint = "https://stroke-preds.herokuapp.com/v1/models/stroke-model:predict"
    response = requests.post(endpoint, data=json_data)

    prediction = response.json()
    return prediction

In [173]:
make_predictions(inputs)

Male
67.0
0
1
Yes
Private
Urban
228.69
36.6
formerly smoked
{'gender': bytes_list {
  value: "Male"
}
, 'age': float_list {
  value: 67.0
}
, 'hypertension': int64_list {
  value: 0
}
, 'heart_disease': int64_list {
  value: 1
}
, 'ever_married': bytes_list {
  value: "Yes"
}
, 'work_type': bytes_list {
  value: "Private"
}
, 'Residence_type': bytes_list {
  value: "Urban"
}
, 'avg_glucose_level': float_list {
  value: 228.69000244140625
}
, 'bmi': float_list {
  value: 36.599998474121094
}
, 'smoking_status': bytes_list {
  value: "formerly smoked"
}
}
[{'examples': {'b64': 'CvsBChUKDGh5cGVydGVuc2lvbhIFGgMKAQAKDwoDYm1pEggSBgoEZmYSQgolCg5zbW9raW5nX3N0YXR1cxITChEKD2Zvcm1lcmx5IHNtb2tlZAoYCgl3b3JrX3R5cGUSCwoJCgdQcml2YXRlCg8KA2FnZRIIEgYKBAAAhkIKFwoMZXZlcl9tYXJyaWVkEgcKBQoDWWVzChYKDWhlYXJ0X2Rpc2Vhc2USBRoDCgEBCh0KEWF2Z19nbHVjb3NlX2xldmVsEggSBgoEpLBkQwobCg5SZXNpZGVuY2VfdHlwZRIJCgcKBVVyYmFuChIKBmdlbmRlchIICgYKBE1hbGU='}}]


{'error': "Name: <unknown>, Key: bmi, Index: 0.  Data types don't match. Data type: float but expected type: string\n\t [[{{function_node __inference_serve_tf_examples_fn_198716}}{{node ParseExample/ParseExampleV2}}]]"}